In [2]:
import pyprob
import pyprob.diagnostics

In [3]:
model = pyprob.RemoteModel('tcp://localhost:2345')

In [4]:
prior = model.prior_traces(num_traces=1)

Time spent  | Time remain.| Progress             | Trace | Traces/sec
ppx (Python): zmq.REQ socket connecting to server tcp://localhost:2345  
ppx (Python): This system        : pyprob 0.13.2.dev10
ppx (Python): Connected to system: pyprob_cpp 0.1.9 (master:16d27c2)
ppx (Python): Model name         : OpenMalaria probprog


RuntimeError: ppx (Python): Sample from an unexpected distribution requested.

In [4]:
trace=prior._values[0]

In [5]:
var = trace.variables

In [7]:
len(trace.variables_dict_address_base) # only 12 things are being called recursively. 
# len(trace.variables_dict_address)

12

In [8]:
trace.execution_time_sec

91.89025807380676

In [9]:
pyprob.diagnostics.graph(prior, file_name="./test.pdf")